In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml.gz')
model

Name,iML1515
Memory address,0x023e277c8f60
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [2]:
from cobra import Reaction, Metabolite
sucd = Reaction('SUCD') #CoA-dependent succinate semialdehyde dehydrogenase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

sucd.add_metabolites({model.metabolites.h_c: 1,
                              model.metabolites.nadh_c: -1,
                              model.metabolites.succoa_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model.metabolites.coa_c: 1,
                              model.metabolites.nad_c: 1
                             })
print(sucd.build_reaction_string())
model.add_reactions([sucd])
model.reactions.SUCD

nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c


Reaction identifier,SUCD
Name,
Memory address,0x023e30d7a4a8
Stoichiometry,nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA --> Coenzyme A + H+ + Nicotinamide adenine dinucleotide +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [3]:
from cobra import Reaction, Metabolite
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model.metabolites.h_c: 1,
                              model.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model.add_reactions([fhbd])
model.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x023e30d7a710
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [4]:
from cobra import Reaction, Metabolite
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model.metabolites.accoa_c: -1,
                              model.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model.add_reactions([hbd_coa_t])
model.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x023e30d7a518
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [5]:
from cobra import Reaction, Metabolite
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model.metabolites.h_c: 1,
                              model.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model.metabolites.nad_c: 1,
                              model.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model.add_reactions([hbd_coa_r])
model.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x023e30d62b38
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [8]:
from cobra import Reaction, Metabolite
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model.metabolites.h_c: 1,
                              model.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model.add_reactions([adh])
model.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x023e30d4c048
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model.metabolites.bdo_c: -1})
model.add_reaction(bdo_exchange)
model.add_boundary(model.metabolites.bdo_c, type='demand')

Reaction identifier,DM_bdo_c
Name,demand
Memory address,0x023e7855e0f0
Stoichiometry,bdo_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [10]:
with model:
    model.objective=model.reactions.ADH
    print(model.optimize().objective_value)

9.605894736842117
